# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [3]:
import numpy as np
import os
!pip install imageio pillow
import imageio
from PIL import Image
import datetime
import os
from PIL import Image
!pip install opencv-python


#write your model here

import pathlib
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import PIL
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Conv3D,MaxPooling3D,Flatten, Dense,Rescaling,Activation,BatchNormalization,Dropout,RandomFlip,RandomRotation
from tensorflow.keras.regularizers import l2


You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [4]:
import cv2

We set the random seed so that the results don't vary drastically.

In [5]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(42)


In [6]:
##from google.colab import drive
#drive.mount('/content/drive')

In [7]:
import os

# List uploaded files
uploaded_files = os.listdir('/home/dl_content/Upgrad DL/Gesture Recognition Assignment')
print(uploaded_files)

# Verify the file extension
zip_file_path = '/home/dl_content/Upgrad DL/Gesture Recognition Assignment/Project_data.zip'  # Ensure this matches the uploaded file name
print(f"Checking file: {zip_file_path}")

# Check if the file is a valid ZIP file
import zipfile

try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        print(f"File {zip_file_path} is a valid zip file.")
except zipfile.BadZipFile:
    print(f"File {zip_file_path} is not a valid zip file.")


['Starter_Code_Gesture_Recognition.ipynb', '.ipynb_checkpoints', 'Project_data.zip', 'Neural_Nets_Project_Starter_Code (6).ipynb']
Checking file: /home/dl_content/Upgrad DL/Gesture Recognition Assignment/Project_data.zip
File /home/dl_content/Upgrad DL/Gesture Recognition Assignment/Project_data.zip is a valid zip file.


In [8]:
import shutil

zip_file_path = '/home/dl_content/Upgrad DL/Gesture Recognition Assignment/Project_data.zip'  # Adjust the filename if needed
if os.path.exists('/home/dl_content/Upgrad DL/Gesture Recognition Assignment/Project_data'):
  shutil.rmtree('/home/dl_content/Upgrad DL/Gesture Recognition Assignment/Project_data')
extracted_folder_path = '/home/dl_content/Upgrad DL/Gesture Recognition Assignment/Project_data/'  # You can change this to any folder name

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

print(f"Folder extracted to: {extracted_folder_path}")


Folder extracted to: /home/dl_content/Upgrad DL/Gesture Recognition Assignment/Project_data/


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [9]:
train_doc = np.random.permutation(open('/home/dl_content/Upgrad DL/Gesture Recognition Assignment/Project_data/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/dl_content/Upgrad DL/Gesture Recognition Assignment/Project_data/Project_data/val.csv').readlines())
batch_size = 50
#print(train_doc)

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [11]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = list(range(30))   #create a list of image numbers you want to use for a particular video, incase if you want conclusion from 15 itself u can use 15
    while True:
        t = np.random.permutation(folder_list)

        num_batches = len(t) // batch_size # calculate the number of batches = 6
        #print(t=663)
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,30,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                #print(imgs = 30)
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image_path = (source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    #print(type(image_path),image_path)
                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = cv2.imread(image_path)

                    # Define the target size
                    target_height = 100
                    target_width = 100

                    # Get the original dimensions
                    original_height, original_width = image.shape[:2]
                    # Check if the image is larger than the target size
                    if original_height > target_height or original_width > target_width:
                        # Resize the image to fit the target size (without distortion)
                        scale_width = target_width / original_width
                        scale_height = target_height / original_height
                        scale = min(scale_width, scale_height)  # Use the smaller scale factor

                        new_width = int(original_width * scale)
                        new_height = int(original_height * scale)
                        resized_image = cv2.resize(image, (new_width, new_height))

                        # Create a blank canvas with the target size
                        padded_image = np.zeros((target_height, target_width, 3), dtype=np.uint8)

                        # Calculate the offset to center the resized image on the canvas
                        y_offset = (target_height - new_height) // 2
                        x_offset = (target_width - new_width) // 2

                        # Place the resized image on the canvas
                        padded_image[y_offset:y_offset+new_height, x_offset:x_offset+new_width] = resized_image

                    else:
                        # If the image is smaller, pad it to the target size
                        pad_top = (target_height - original_height) // 2
                        pad_bottom = target_height - original_height - pad_top
                        pad_left = (target_width - original_width) // 2
                        pad_right = target_width - original_width - pad_left

                        # Add padding to the image
                        padded_image = cv2.copyMakeBorder(image, pad_top, pad_bottom, pad_left, pad_right,
                                      cv2.BORDER_CONSTANT, value=(0, 0, 0))


                    # Optionally, save the processed image (either resized or cropped)
                    cv2.imwrite(image_path, padded_image)

                    #resized_image = cv2.resize(image,(200,200))
                    image_array = np.array(padded_image)
                    batch_data[folder,idx,:,:,:] = image_array / 255.0
                    ##batch_data[folder,idx,:,:,1] = image_array / 255.0
                    #batch_data[folder,idx,:,:,2] = image_array / 255.0
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
            #print(batch_data,batch_labels)



        ##code draw last thirteen folder images
        leftoverfoldersize = len(t) % batch_size
        folder2=t[-leftoverfoldersize:]
        for folderindex in range(leftoverfoldersize):
            imagelist=os.listdir(source_path+'/'+ folder2[folderindex].split(';')[0])
            for idx,item in enumerate(img_idx):
                image_path = (source_path+'/'+ folder2[folderindex].strip().split(';')[0]+'/'+imagelist[item])
                #print(type(image_path),image_path)
                #crop the images and resize them. Note that the images are of 2 different shape
                #and the conv3D will throw error if the inputs in a batch have different shapes
                image = cv2.imread(image_path)

                # Define the target size
                target_height = 100
                target_width = 100

                # Get the original dimensions
                original_height, original_width = image.shape[:2]
                # Check if the image is larger than the target size
                if original_height > target_height or original_width > target_width:
                    # Resize the image to fit the target size (without distortion)
                    scale_width = target_width / original_width
                    scale_height = target_height / original_height
                    scale = min(scale_width, scale_height)  # Use the smaller scale factor

                    new_width = int(original_width * scale)
                    new_height = int(original_height * scale)
                    resized_image = cv2.resize(image, (new_width, new_height))

                    # Create a blank canvas with the target size
                    padded_image = np.zeros((target_height, target_width, 3), dtype=np.uint8)

                    # Calculate the offset to center the resized image on the canvas
                    y_offset = (target_height - new_height) // 2
                    x_offset = (target_width - new_width) // 2

                    # Place the resized image on the canvas
                    padded_image[y_offset:y_offset+new_height, x_offset:x_offset+new_width] = resized_image

                else:
                    # If the image is smaller, pad it to the target size
                    pad_top = (target_height - original_height) // 2
                    pad_bottom = target_height - original_height - pad_top
                    pad_left = (target_width - original_width) // 2
                    pad_right = target_width - original_width - pad_left

                    # Add padding to the image
                    padded_image = cv2.copyMakeBorder(image, pad_top, pad_bottom, pad_left, pad_right,
                                      cv2.BORDER_CONSTANT, value=(0, 0, 0))


                    # Optionally, save the processed image (either resized or cropped)
                    cv2.imwrite(image_path, padded_image)

                    #resized_image = cv2.resize(image,(200,200))
                    image_array = np.array(padded_image)
                    batch_data[folder,idx,:,:,:] = image_array / 255.0
                    ##batch_data[folder,idx,:,:,1] = image_array / 255.0
                    #batch_data[folder,idx,:,:,2] = image_array / 255.0
            batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1

        yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

    


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [12]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/dl_content/Upgrad DL/Gesture Recognition Assignment/Project_data/Project_data/train'
val_path = '/home/dl_content/Upgrad DL/Gesture Recognition Assignment/Project_data/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [13]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
#from keras.layers.convolutional import Conv3D, MaxPooling3D
from tensorflow.keras.layers import Conv3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers



model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(30,100,100,3)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(5, activation='softmax'))



2025-01-23 12:31:59.102964: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2025-01-23 12:31:59.103060: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14800 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:1d:00.0, compute capability: 7.5


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [14]:
optimiser = 'adam'
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 28, 98, 98, 32)    2624      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 14, 49, 49, 32)   0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 12, 47, 47, 64)    55360     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 6, 23, 23, 64)    0         
 3D)                                                             
                                                                 
 flatten (Flatten)           (None, 203136)            0         
                                                                 
 dense (Dense)               (None, 256)               5

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [15]:
train_generator=generator(train_path, train_doc, batch_size)
print(type(train_generator))
val_generator = generator(val_path, val_doc, batch_size)

<class 'generator'>


In [16]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.keras'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',save_freq='epoch')


LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001, verbose=1) # Define a learning rate scheduler

callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [17]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [18]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None,
           initial_epoch=0)

Source path =  /home/dl_content/Upgrad DL/Gesture Recognition Assignment/Project_data/Project_data/train ; batch size = 50
Epoch 1/20


2025-01-23 12:32:04.726704: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302


14/14 [==============================] - ETA: 0s - loss: 27.3335 - categorical_accuracy: 0.2214Source path =  /home/dl_content/Upgrad DL/Gesture Recognition Assignment/Project_data/Project_data/val ; batch size = 50

Epoch 00001: saving model to model_init_2025-01-2312_31_58.157348/model-00001-27.33346-0.22143-1.49587-0.30000.keras
14/14 [==============================] - 56s 4s/step - loss: 27.3335 - categorical_accuracy: 0.2214 - val_loss: 1.4959 - val_categorical_accuracy: 0.3000 - lr: 0.0010
Epoch 2/20
14/14 [==============================] - ETA: 0s - loss: 1.2710 - categorical_accuracy: 0.4757
Epoch 00002: saving model to model_init_2025-01-2312_31_58.157348/model-00002-1.27103-0.47571-1.37740-0.46000.keras
14/14 [==============================] - 31s 2s/step - loss: 1.2710 - categorical_accuracy: 0.4757 - val_loss: 1.3774 - val_categorical_accuracy: 0.4600 - lr: 0.0010
Epoch 3/20
14/14 [==============================] - ETA: 0s - loss: 0.7173 - categorical_accuracy: 0.7771
Epoch

##### Will bUild model with CNN+RNN architecture

In [19]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, TimeDistributed, GRU

# Define the model
modellstm = Sequential()

# Add CNN layers (feature extraction part)
modellstm.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(30, 100, 100, 3)))  # TimeDistributed wrapper to apply Conv2D on each frame
modellstm.add(TimeDistributed(MaxPooling2D((2, 2))))

modellstm.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
modellstm.add(TimeDistributed(MaxPooling2D((2, 2))))

modellstm.add(TimeDistributed(Flatten()))  # Flatten the features from each frame

# Add RNN layers (sequence modeling part)
modellstm.add(LSTM(128, return_sequences=False))  
modellstm.add(Dense(5, activation='softmax'))  


In [20]:
optimiser = 'adam'
modellstm.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (modellstm.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 30, 98, 98, 32)   896       
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 30, 49, 49, 32)   0         
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 30, 47, 47, 64)   18496     
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 30, 23, 23, 64)   0         
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 30, 33856)       

In [21]:
model_name = 'modellstm_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.keras'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',save_freq='epoch')


LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001, verbose=1) # Define a learning rate scheduler

callbacks_list = [checkpoint, LR]

In [22]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [23]:
modellstm.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None,
           initial_epoch=0)

Epoch 1/20
14/14 [==============================] - ETA: 0s - loss: 1.9557 - categorical_accuracy: 0.2429
Epoch 00001: saving model to modellstm_init_2025-01-2312_31_58.157348/model-00001-1.95567-0.24286-1.64826-0.16000.keras
14/14 [==============================] - 34s 2s/step - loss: 1.9557 - categorical_accuracy: 0.2429 - val_loss: 1.6483 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 2/20
14/14 [==============================] - ETA: 0s - loss: 1.6051 - categorical_accuracy: 0.2543
Epoch 00002: saving model to modellstm_init_2025-01-2312_31_58.157348/model-00002-1.60511-0.25429-1.58912-0.27000.keras
14/14 [==============================] - 30s 2s/step - loss: 1.6051 - categorical_accuracy: 0.2543 - val_loss: 1.5891 - val_categorical_accuracy: 0.2700 - lr: 0.0010
Epoch 3/20
14/14 [==============================] - ETA: 0s - loss: 1.5720 - categorical_accuracy: 0.3000
Epoch 00003: saving model to modellstm_init_2025-01-2312_31_58.157348/model-00003-1.57203-0.30000-1.56081-0.30

In [24]:
##rm -rf ~/.local/share/Trash/*
